Author: Zachary Flamholz  
Date: 06-2018  
Database: https://www.yeastgenome.org/  
Data: http://geneontology.org/gene-associations/gene_association.sgd.gz  
Companion file: ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Fungi/Saccharomyces_cerevisiae.gene_info.gz, http://snapshot.geneontology.org/ontology/go-basic.obo    

In [1]:
import numpy as np
import pandas as pd
import sys, datetime
import goenrich
import scipy.stats as stat

# Versions of Modules in Use

In [75]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget 

Software versions
Python 2.7.15 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 5.7.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.3
pandas 0.23.0
clustergrammer_widget The 'clustergrammer_widget' distribution was not found and is required by the application
Thu Jun 21 17:53:04 2018 EDT

## read in data

In [8]:
df = pd.read_csv('input/gene_association.sgd', sep='~', skiprows=18, header=None)

In [9]:
df.head()

,0
0,SGD\tS000002318\tSTE7\t\tGO:0000165\tPMID:8455...
1,SGD\tS000002318\tSTE7\t\tGO:0000165\tPMID:8455...
2,SGD\tS000002318\tSTE7\t\tGO:0000165\tPMID:8668...
3,SGD\tS000002318\tSTE7\t\tGO:0000165\tPMID:8668...
4,SGD\tS000002318\tSTE7\t\tGO:0000187\tPMID:8384...


In [12]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
matrix = np.chararray((df.shape[0], 17), itemsize=150, unicode=True)

for i, row in enumerate(df.itertuples()):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    lst = row[1].split('\t')

    matrix[i, :] = lst

df_clean = pd.DataFrame(data=matrix)

In [13]:
df_clean.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,SGD,S000002318,STE7,,GO:0000165,PMID:8455599|SGD_REF:S000042071,IDA,,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
1,SGD,S000002318,STE7,,GO:0000165,PMID:8455599|SGD_REF:S000042071,IGI,SGD:S000004354,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
2,SGD,S000002318,STE7,,GO:0000165,PMID:8668180|SGD_REF:S000041791,IPI,SGD:S000000112,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
3,SGD,S000002318,STE7,,GO:0000165,PMID:8668180|SGD_REF:S000041791,IPI,SGD:S000003272,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
4,SGD,S000002318,STE7,,GO:0000187,PMID:8384702|SGD_REF:S000045748,IDA,,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,"has_direct_input(UniProtKB:P16892),part_of(GO:...",
5,SGD,S000002318,STE7,,GO:0000187,PMID:8384702|SGD_REF:S000045748,IMP,,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
6,SGD,S000003664,PBS2,,GO:0000187,PMID:7624781|SGD_REF:S000051778,IGI,SGD:S000000669,P,MAP kinase kinase of the HOG signaling pathway,YJL128C|HOG4|SFS4|SSK4|mitogen-activated prote...,gene,taxon:559292,20171214,SGD,"has_input(UniProtKB:P32485),part_of(GO:0071474)",
7,SGD,S000003664,PBS2,,GO:0000187,PMID:7624781|SGD_REF:S000051778,IGI,SGD:S000005314,P,MAP kinase kinase of the HOG signaling pathway,YJL128C|HOG4|SFS4|SSK4|mitogen-activated prote...,gene,taxon:559292,20171214,SGD,"has_input(UniProtKB:P32485),part_of(GO:0071474)",
8,SGD,S000004354,STE11,,GO:0000165,PMID:8455599|SGD_REF:S000042071,IGI,SGD:S000002318,P,Signal transducing MEK kinase,YLR362W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
9,SGD,S000000537,STE50,,GO:0000165,PMID:9742096|SGD_REF:S000065130,IGI,"SGD:S000005314,SGD:S000000669",P,Adaptor protein for various signaling pathways,YCL032W,gene,taxon:559292,20171215,SGD,part_of(GO:0071474),


In [20]:
df_clean.iloc[:, 16].unique()

array([u''], dtype=object)

In [14]:
columns = ["DB", "DB gene ID", "Gene symbol", "Qualifier", "GO ID", "Reference", "Evidence code", "Evidence from", "GO class", "attribute", "Locus tag", "gene/protein", "tax id", "date", "Assigned by", "additional information", "empty"]

In [21]:
df_clean.columns = columns

In [22]:
df_clean.head()

,DB,DB gene ID,Gene symbol,Qualifier,GO ID,Reference,Evidence code,Evidence from,GO class,attribute,Locus tag,gene/protein,tax id,date,Assigned by,additional information,empty
0,SGD,S000002318,STE7,,GO:0000165,PMID:8455599|SGD_REF:S000042071,IDA,,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
1,SGD,S000002318,STE7,,GO:0000165,PMID:8455599|SGD_REF:S000042071,IGI,SGD:S000004354,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
2,SGD,S000002318,STE7,,GO:0000165,PMID:8668180|SGD_REF:S000041791,IPI,SGD:S000000112,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
3,SGD,S000002318,STE7,,GO:0000165,PMID:8668180|SGD_REF:S000041791,IPI,SGD:S000003272,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,part_of(GO:0000749),
4,SGD,S000002318,STE7,,GO:0000187,PMID:8384702|SGD_REF:S000045748,IDA,,P,Signal transducing MAP kinase kinase,YDL159W|mitogen-activated protein kinase kinas...,gene,taxon:559292,20171213,SGD,"has_direct_input(UniProtKB:P16892),part_of(GO:...",


In [28]:
df_process = df_clean[df_clean["GO class"] == 'P'].copy()

In [29]:
df_process.shape

(43136, 17)

In [30]:
# remove any annotation assigned by electronic matching and with the NOT qualifier which is used to specify a gene is not associated with a term
df_process = df_process[df_process["Evidence code"] != 'IEA']
df_process = df_process[df_process["Qualifier"] != 'NOT']

In [31]:
df_process = df_process[['Gene symbol', 'GO ID']]
df_process.reset_index(inplace=True)
df_process = df_process.drop(columns = 'index')

In [32]:
df_process.shape

(28256, 2)

## load GO digraph

In [33]:
digraph = goenrich.obo.ontology('input/go-basic.ob')

## only keep GO IDs with depth =>4, replace others with NaN and then remove those rows

In [34]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 

lst = []

for i,index in enumerate(df_process.index):
    
    progressPercent = ((i+1)/len(df_process.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_process.index)))
    sys.stdout.flush()
    
    term = df_process.loc[index, 'GO ID']
    if term in digraph.node:
        if digraph.node[term]['depth'] >= 4:
            lst.append(term)
        else:
            lst.append(np.nan)
    else:
        lst.append(np.nan)
           
df_process['GO ID'] = lst

In [35]:
df_process.shape

(28256, 2)

In [36]:
df_process.dropna(inplace=True)

In [37]:
df_process.shape

(24288, 2)

In [38]:
df_process.head()

,Gene symbol,GO ID
0,STE7,GO:0000165
1,STE7,GO:0000165
2,STE7,GO:0000165
3,STE7,GO:0000165
4,STE7,GO:0000187


## term propagation-propergate child gene term relationships to parent terms

In [40]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst1 = []
lst2 = []

for i,index in enumerate(df_process.index):
    
#     progressPercent = ((i+1)/len(df.index))*100

#     sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
#     sys.stdout.flush()
    
#     term = df.loc[index, 'GO ID']
#     for parent in digraph.predecessors(term):
#         if parent in digraph.node:
#             if digraph.node[parent]['depth'] >= 4:
#                 lst1.append(df.loc[index, 'DB Object Symbol'])
#                 lst2.append(parent)
#                 print(term, parent)
    term = df_process.loc[index, 'GO ID']
    for parent in digraph.successors(term):
        if parent in digraph.node:
            if digraph.node[parent]['depth'] >= 4:
                lst1.append(df_process.loc[index, 'Gene symbol'])
                lst2.append(parent)
#                 print(term, parent)


temp = pd.DataFrame()
temp['Gene symbol'] = lst1
temp['GO ID']  = lst2
df_process = pd.concat([df_process, temp])
df_process.reset_index(inplace=True)
df_process = df_process.drop(columns = 'index')

In [41]:
df_process.shape

(65864, 2)

In [42]:
df_process.head()

,Gene symbol,GO ID
0,STE7,GO:0000165
1,STE7,GO:0000165
2,STE7,GO:0000165
3,STE7,GO:0000165
4,STE7,GO:0000187


## map GO ID to descriptive name

In [43]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst = []

for i,index in enumerate(df_process.index):
    
    progressPercent = ((i+1)/len(df_process.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_process.index)))
    sys.stdout.flush()
    
    lst.append(str(digraph.node[df_process.loc[index, 'GO ID']]['name'])+' '+ '('+str(df_process.loc[index, 'GO ID'])+')')
    
df_process['GO ID'] = lst

In [44]:
df_process.head()

,Gene symbol,GO ID
0,STE7,MAPK cascade (GO:0000165)
1,STE7,MAPK cascade (GO:0000165)
2,STE7,MAPK cascade (GO:0000165)
3,STE7,MAPK cascade (GO:0000165)
4,STE7,activation of MAPK activity (GO:0000187)


In [45]:
df_process.drop_duplicates(inplace=True)

In [46]:
df_process.head()

,Gene symbol,GO ID
0,STE7,MAPK cascade (GO:0000165)
4,STE7,activation of MAPK activity (GO:0000187)
6,PBS2,activation of MAPK activity (GO:0000187)
8,STE11,MAPK cascade (GO:0000165)
9,STE50,MAPK cascade (GO:0000165)


In [48]:
df_process.reset_index(inplace=True)

In [49]:
df_process.head()

,index,Gene symbol,GO ID
0,0,STE7,MAPK cascade (GO:0000165)
1,4,STE7,activation of MAPK activity (GO:0000187)
2,6,PBS2,activation of MAPK activity (GO:0000187)
3,8,STE11,MAPK cascade (GO:0000165)
4,9,STE50,MAPK cascade (GO:0000165)


In [50]:
df_process = df_process.drop(columns = 'index')

In [51]:
df_process.head()

,Gene symbol,GO ID
0,STE7,MAPK cascade (GO:0000165)
1,STE7,activation of MAPK activity (GO:0000187)
2,PBS2,activation of MAPK activity (GO:0000187)
3,STE11,MAPK cascade (GO:0000165)
4,STE50,MAPK cascade (GO:0000165)


In [52]:
df_process.shape

(37399, 2)

In [56]:
len(df_process.iloc[:, 1].unique())

4111

## build the protein-coding gene list

In [57]:
sCerevisiae_geneInfo = pd.read_csv("input/Saccharomyces_cerevisiae.gene_info", sep="\t")

In [58]:
sCerevisiae_geneInfo.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,4932,2828223,NEWENTRY,-,-,-,-,-,Record to support submission of GeneRIFs for a...,other,-,-,-,-,20180616,-
1,4932,24573110,atp6,ACI60_gp06,-,-,MT,-,Atp6,protein-coding,-,-,-,-,20180130,-
2,4932,24573111,ACI60_gt03,ACI60_gt03,-,-,MT,-,tRNA,tRNA,-,-,-,-,20150619,-
3,4932,24573112,ACI60_gt04,ACI60_gt04,-,-,MT,-,tRNA,tRNA,-,-,-,-,20150619,-
4,4932,24573113,ACI60_gr01,ACI60_gr01,-,-,MT,-,large subunit ribosomal RNA,rRNA,-,-,-,-,20150619,-


In [59]:
sCerevisiae_geneInfo.shape

(6482, 16)

In [64]:
sCerevisiae_proteinCoding = sCerevisiae_geneInfo[sCerevisiae_geneInfo["type_of_gene"] == "protein-coding"]

In [65]:
sCerevisiae_proteinCoding.shape

(6010, 16)

In [66]:
len(sCerevisiae_proteinCoding[sCerevisiae_proteinCoding["description"] == "hypothetical protein"])

719

## only keep gene symbolys that are in the S cerevisiae table

In [67]:
df_process_protein = df_process.loc[df_process["Gene symbol"].isin(sCerevisiae_proteinCoding["Symbol"].unique())]

In [68]:
df_process_protein.shape

(36009, 2)

In [69]:
filename = 'process_gene_ontology_SGD_greater4_%s.gmt'% str(datetime.date.today())[0:7].replace('-', '_')
file = open(filename,'w+') 
terms = df_process_protein["GO ID"].unique()
for i,term in enumerate(terms):
    
    progressPercent = ((i+1)/len(terms))*100
    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(terms)))
    sys.stdout.flush()
            
    df_byTerm = df_process_protein.loc[df_process_protein["GO ID"] == term]
    
    if df_byTerm.shape[0] > 4:
        # split splice variant names
        split_splice = lambda x: x.split('.')[0]
        df_byTerm["Gene symbol"] = df_byTerm["Gene symbol"].apply(split_splice)
        
        
        if len(df_byTerm["Gene symbol"].unique()) > 4:
            file.write("%s\t" % term)
            file.write("\t")
            genes = df_byTerm["Gene symbol"].unique()
    
            for gene in genes:
                file.write("%s\t" % gene)
            file.write("\n")
        
file.close()

/Users/maayanlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## get stats for gmt

In [70]:
def getTermStats(loaded_gmt, geneInfoTable, num_samplings, num_genes_in_sampling) :
    # get the relevant terms and set a dictionary for them
    term_set = loaded_gmt.iloc[:,0].apply(lambda x: x.split("\t")[0])
    term_genes_dict = {term_set[i]: loaded_gmt.iloc[i,0].split("\t")[2:] for i in range(0, loaded_gmt.shape[0])}
    term_rank_dict = {k: [] for k in term_set}
    term_stats_dict = {k: [] for k in term_set}
    
    # get the total number of genes in the organism
    n_genes = len(geneInfoTable["Symbol"].unique())
    
    # number of genes per sampling, number of times to sample
    genes_in_sampleing = num_genes_in_sampling
    num_samplings = num_samplings
    
    for i,x in enumerate(range(0,num_samplings)):
        
        progressPercent = ((i+1)/len(range(0,num_samplings)))*100
        
        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(range(0,num_samplings))))
        sys.stdout.flush()
            
        
        randomSet = set(geneInfoTable["Symbol"].sample(genes_in_sampleing).apply(lambda x: x.split('.')[0]))
        
        # initialize an array to hold the pvalue for each term
        pvals = np.array([])
        for key in term_rank_dict.keys():
            
            termSet = set(term_genes_dict[key])
            # calculate p value using fisher exact test
            # implemented using the formula found in the GeneOverap bioconductor package for R
            pval_term = stat.fisher_exact([[n_genes - len(termSet.union(randomSet)), len(randomSet.difference(termSet))], [len(termSet.difference(randomSet)), len(termSet.intersection(randomSet))]])[1]
            pvals = np.append(pvals, [pval_term])
        
        # sort the pvals and add the rank to the term_rank dict
        sorted_pvals = pvals.argsort()
        
        # find the rank for each term by sorting the pvals array and getting the index of the key position in the
        # sorted list. Need to +1 because the index begins with 0
        
        for j,key in enumerate(term_rank_dict.keys()):
            term_rank_dict[key].append(np.where(sorted_pvals == j)[0][0] + 1)
            
    for key in term_rank_dict:
        term_stats_dict[key] = [np.mean(term_rank_dict[key]), np.std(term_rank_dict[key])]
            
    
    
    return term_stats_dict

In [71]:
process_GO_gmt = pd.read_csv("output/process_gene_ontology_SGD_greater4_2018_06.gmt", sep="~", header=None)
type(process_GO_gmt)

pandas.core.frame.DataFrame

In [72]:
iterations_200 = getTermStats(process_GO_gmt, sCerevisiae_proteinCoding, num_samplings=200, num_genes_in_sampling=300)

In [73]:
df_200_iterations = pd.DataFrame.from_dict(iterations_200, orient='index', columns = ['mean', 'sd'])

In [74]:
df_200_iterations.to_csv('process_gene_ontology_sdg_greater4_2018_06_stats.tsv', sep='\t', header=False)